# HW: DB in Python

1. Create classes for tables Subject and Student

> Add models for student, subject and student_subject from previous lessons in SQLAlchemy.

In [ ]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship

Base = declarative_base()

class Student(Base):
    __tablename__ = 'student'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    subject_id = Column(Integer, ForeignKey('subject.id'))

    # Define a relationship to the Subject table
    subject = relationship('Subject', back_populates='students')

    def __str__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}. Subject_id: {self.subject_id}'

    def __repr__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}. Subject_id: {self.subject_id}'


class Subject(Base):
    __tablename__ = 'subject'

    id = Column(Integer, primary_key=True)
    subject_name = Column(String)

    # Define a back reference to the Student table
    students = relationship('Student', back_populates='subject')

    def __str__(self):
        return f'This is {self.subject_name} subject. Id: {self.id}'

    def __repr__(self):
        return f'This is {self.subject_name} subject. Id: {self.id}'

2. Get keys for database from .env_example

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(".env.example")

PASSWORD = os.getenv("PASSWORD", " ")
DATABASE = os.getenv("DATABASE", " ")
USER =  os.getenv("USER", " ")

print(PASSWORD, DATABASE, USER)

3. Connect to the database

In [ ]:
from sqlalchemy import create_engine

DATABASE_URI = 'postgresql://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(
    DATABASE_URI.format(
        host='localhost',
        database=DATABASE,
        user=USER,
        password=PASSWORD,
        port=5432,
    )
)

4. Start the session

In [ ]:
from sqlalchemy.orm import sessionmaker

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

5. Insert the values

In [ ]:
# Insert values into the table Subject
subject_names = ['English', 'Spanish', 'Chinese', 'Math', 'History', 'Science']

for sub_name in subject_names:
    subject = Subject(subject_name=sub_name)
    session.add(subject)

session.commit()

In [ ]:
session.query(Subject).all()

In [ ]:
# Insert values into the table Student
from random import randint, choice

random_names = ['Joe', 'Winston', 'Benedict', 'Andrea', 'Fillipe', 'Iria', 'Eric', 'Joan', 'Pablo', 'Brian', 'Timothy', 'Lee']

for _ in range(10):
    student = Student(name=choice(random_names), age=randint(18, 25), subject_id=randint(1,6))
    session.add(student)

session.commit()

In [ ]:
session.query(Student).all()

6. Join tables Student and Subject into the student_subject table. Add it to the database

In [ ]:
# joining tables Student and Subject

result = session.query(Student, Subject).join(Subject).all()

for student, subject in result:
    print(student.id, student.name, student.age, subject.id, subject.subject_name)

In [ ]:
# Trying to populate the table StudentSubject from the join result query -- unsuccessful

class StudentSubjects(Base):
    __tablename__ = 'student_subjects'

    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('student.id'))
    student_name = Column(Integer, ForeignKey('student.name'))
    student_age = Column(Integer, ForeignKey('student.age'))
    subject_name = Column(Integer, ForeignKey('subject.subject_name'))

    student = relationship('Student', foreign_keys=[student_id])
    subject = relationship('Subject', foreign_keys=[subject_name])


for student, subject in result:
    student_subject_obj = StudentSubjects(student_id=student.id, student_name=student.name, student_age=student.age, subject_name=subject.subject_name)
    session.add(student_subject_obj)

session.query(StudentSubjects).all()

# session.commit()

> Find all students` name that visited 'English' classes.

In [ ]:
session.query(Student).filter(Student.subject_id == '1').all()